In [3]:
import torch
from transformers import BertJapaneseTokenizer, BertModel
from transformers import AutoModelForCausalLM, AutoTokenizer, PretrainedConfig, AutoModelForCausalLM


import matplotlib.pyplot as plt
import numpy as np
# https://developer.mamezou-tech.com/blogs/2023/03/26/using-transformer-03/
# https://www.ai-shift.co.jp/techblog/2089

In [4]:
model_name = "../data/model/strf_sonoisa_sentence-bert-base-ja-mean-tokens-v232.75.10"
tokenizer = BertJapaneseTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name, output_attentions=True)

In [71]:
sen = "仙骨部の腫瘍の重症度フォローをお願いします。"
sen = "仙腸関節炎、強直性脊椎炎疑い 腰椎～仙腸関節までお願いします"
# sen = "直腸癌の局所再発に対する切除後。 ストーマ閉鎖後。 下肢への放射線を伴う臀部の痛み。 再発はありますか？ 腰部脊柱管狭窄症ですか？ 精査してください"
#sen = "2/10発症左椎骨動脈解離による脳塞栓の患者｡ 大動脈弓部から頸部動脈のplaque imaging撮像お願いします｡"
sen = "cisに対して2013年子宮両側付属器切除､腟断端 再発に対して放射線治療後｡右腟壁下部~右大陰唇皮下にかけて､径2cm程度の腫瘤を触知｡前医の生検にて high grade intraepithelia l squamous neoplasiaと診断されています｡腫瘤のｻｲｽﾞや広がりについて御高診お願いします｡"
sen = "昨年他院でanaplastic oligodendrogliomaの診断 今回多発性の再発疑い 頭蓋内髄腔播種を疑うため脊髄病変のcheck目的です 頭部plateあり"
input_text = sen
input_ids = tokenizer.encode(input_text, return_tensors='pt')

In [72]:
# Attentionを取得する
outputs = model(input_ids)
attentions = outputs.attentions
# 言語モデルに入力された文に対するAttentionの重みを取得
attention = torch.sum(attentions[-1], dim=1)[0].detach().numpy()

In [73]:
attn = attention[0,:]
attn = attn/attn.max()
tokens = tokenizer.convert_ids_to_tokens(tokenizer.encode(input_text))

print (len(attn), len(tokens))

51 51


In [74]:
for a, t in zip(attn, tokens):
    print(a,t)

0.18954526 [CLS]
0.12903972 昨年
0.11135468 他
0.18506832 院
0.04080171 で
0.1582808 an
0.067936264 ##ap
0.05807882 ##las
0.01232619 ##t
0.06512184 ##ic
0.118905365 o
0.061555468 ##li
0.03134367 ##go
0.019381912 ##de
0.021385398 ##n
0.0234177 ##d
0.027846003 ##ro
0.02705255 ##g
0.02909264 ##li
0.013861548 ##om
0.10537929 ##a
0.11824307 の
0.06603511 診断
0.01758679 今回
0.15634473 多発
0.010893926 性
0.0052205212 の
0.07104178 再発
0.00361242 疑い
0.1346438 頭蓋
0.024140358 内
0.07160173 髄
0.0263462 腔
0.05059225 播
0.06210912 ##種
0.004600583 を
0.035896298 疑
0.041808985 ##う
0.23942669 ため
0.7670161 脊髄
0.31569573 病変
0.099310845 の
0.4715068 ch
0.8040261 ##eck
1.0 目的
0.030996496 です
0.62625563 頭部
0.47669944 pl
0.66069716 ##ate
0.60941124 あり
0.028551903 [SEP]


In [91]:
def highlight(word, attn):
  html_color = '#%02X%02X%02X' % (255, int(255*(1 - attn)), int(255*(1 - attn)))
  return '<span style="background-color: {}">{}</span>'.format(html_color, word)

def mk_html(tokens, attn):

  html = ''
  for i, a in enumerate(attn):
    html += highlight(tokenizer.convert_ids_to_tokens(tokenizer.encode(input_text))[i], a)  
  html += "<br><br>"
  return html


from IPython.display import display, HTML

In [92]:
html_output = mk_html(tokens, attn)
display(HTML(html_output))